In [462]:
import re

In [463]:
f = open("./Utvalda_dokument-294-2016-02-23.txt")
text = f.read()

# Approach 1: parsea ut rubrik, metadata, brödtext för sig

In [436]:
#article_pattern = re.compile(r"(?P<headline>[ \w,\:\!\?\.]+?)\n\-{78,}\n(?P<source>[ \w]+?), (?P<date>\d+\-\d+\-\d+)(?P<author>[\n \w\.\@]+?)(?P<section>Sektion:[ \w\,\d:\.]+)\n(?P<mediatype>[ \w\.]+)\n\n(?P<body>.*?)\={78,}",re.DOTALL)
article_pattern = re.compile(r"(?P<headline>[ \w,\:\gi!\?\.]+?)\n\-{78,}\n(?P<meta>.*?)\n\n(?P<body>.*?)\n\n\={78,}",re.DOTALL)

In [464]:
print(article_pattern.search(text))

<_sre.SRE_Match object; span=(20919, 25797), match='Lycka till, Mr Cameron\n------------------------->


In [465]:
match = article_pattern.search(text)

In [466]:
match.group("headline")

'Lycka till, Mr Cameron'

In [467]:
match.group("meta")

'Dagens Nyheter, 2016-02-22\nSektion: Ledare Sida 4, del 1\nPublicerat i print. '

In [468]:
match.group("body")

'Brexit.\n\nStartskottet har gått. I helgen spikade premiärminister David Cameron datumet för \nden utlovade folkomröstningen om Storbritanniens EU-medlemskap. Den 23 juni avgör \nbritterna sin - och det europeiska samarbetets - framtid. Den uppgörelse om nya \nmedlemsvillkor som EU:s stats- och regeringschefer nådde på fredagskvällen öppnar \nenligt Cameron dörren till den bästa av två världar: Storbritannien kommer \natt sitta i "förarsätet på världens största marknad". Samtidigt slipper landet räddningspaketen \noch euroeländet. Britterna har säkrat en bekväm position i unionen. \nBäst att vi sitter still i båten, slog Cameron fast. Avtalet är fjärran från \nden "långtgående och väsentliga" reform som premiärministern gick in för att tvinga \nfram. Torypartiets ursprungliga, och orimliga, ultimatum har gradvis reducerats \ntill en räcka krav som de andra länderna motvilligt kan tillmötesgå. De fyra \nunderrubrikerna på dagordningen har gällt omförhandling av EU:s inflytande över \nb

In [469]:
article_matches = article_pattern.findall(text)

Och så kollar vi så vår brytpunkt på rubrik-text följt av 78 st ------- ger rätt antal artiklar (294)

In [472]:
len(article_matches)

250

**Men det gör den ju inte!**

Varför?

Jo, en närmare granskning av Retriever-filen visar att de bryter långa rubriker *efter* strecken, så de dubbla \n vi letar efter fungerar inte som avslutning. 

Det är precis såna inkonsistenser i utdatan en inte vill ha från sina data-leverantörer. På detta vis blir inte metadatan enkel att parsea ut, vilket den faktiskt inte var innan heller eftersom det inte alltid finns ett namn och mejladress i metadatan till artikeln. Om det *inte* finns så tas hela raden bort, istället för att vara kvar, men med nån text såsom "NaN" som gör att det går att konsekvent parsea ut varje metadata fält för sig.

Fy skäms Retriever!

# Approach 2: Vi splittar på artikel-slut ("===...") istället

In [473]:
articles = re.split("\={78,}",text)

In [474]:
len(articles)

294

Men varje fil inleds med en listning av alla träffar som vi nu alltså får med i den första artikeln.
Vi får processa den artikeln separat.

Vi kan inte splitta den på -----------------, för då tappar vi den första rubriken som ligger *över* den linjen.

En granskning av textfilen visar att den första artikeln slutar med 3 st \n efter ett datum. Vi håller tummarna att detta är konsekvent i Retrievers txt-filer och splittar på den. Om de inte är konsekventa ens med detta så är det nog bäst att prata med dem om att de fixar till sitt utformat så det blir maskinläsbart ordentligt.

In [459]:
first_article_split = re.split("\d+\-\d+\-\d+\n\n\n", articles[0])

In [460]:
len(first_article_split)

2

Den andra delen, med index 1 är den vi vill ha:

In [478]:
first_article_clean = first_article_split[1] 

In [481]:
# vi kollar för säkerhets skull att det blivit rätt med rubriken
first_article_clean[:100]

'Lycka till, Mr Cameron\n-----------------------------------------------------------------------------'

Så om vi sätter ihop dessa process-steg får vi alltså:

In [482]:
articles = re.split("\={78,}",text)
first_article_split = re.split("\d+\-\d+\-\d+\n\n\n", articles[0])
first_article_clean = first_article_split[1] 

# ta bort den första dåliga artikeln från listan
del articles[0]
# sätt dit den tvättade artikeln på slutet (default) av listan
articles.append(first_article_good)

Vill en ta bort bildtexter och copyrigth info också skulle du kunna pröva med att köra regular expression på varje artikel för sig.

Dvs först parsea ut artiklarna.

Därefter loopa igenom listan med artiklar och leta efter mönster så är ungefär:
    
```
(Bildtext: \w+|@ \w+)
```

Det kan vara ett rimligt nästa steg efter att ni ändrar i funktionen som processar varje fil i ert script. Det är mindre komplext (och därför bättre) att inte använda NLTK och tokenizering om det inte verkligen behövs, så öpnna respektive fil för läsning som vanligt med

```
f = open(path + filename),"r",encoding="utf-8")
text = f.read()
```

och därefter stoppa in regex-koden ovan.
